# 1. KNN Determines Whether Negative Numbers are Possible

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

### 1.1 Data
- 사용할 데이터는 음수가 가능한지를 판단하는 데이터

In [2]:
water = pd.read_csv('../02. Data/water_potability.csv')

data = water.drop(["Potability"], axis=1)
label = water["Potability"]

In [3]:
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075
...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658


In [4]:
label

0       0
1       0
2       0
3       0
4       0
       ..
3271    1
3272    1
3273    1
3274    1
3275    1
Name: Potability, Length: 3276, dtype: int64

### 1.2 EDA

In [5]:
data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000


In [6]:
data.isna().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
dtype: int64

### 1.3 Preprocess

In [7]:
# 결측치가 있는 행 제거
data.isna().sum(axis=1)

0       1
1       1
2       1
3       0
4       0
       ..
3271    0
3272    2
3273    1
3274    1
3275    1
Length: 3276, dtype: int64

In [8]:
na_cnt = data.isna().sum(axis=1)
na_cnt

0       1
1       1
2       1
3       0
4       0
       ..
3271    0
3272    2
3273    1
3274    1
3275    1
Length: 3276, dtype: int64

In [9]:
drop_idx = na_cnt.loc[na_cnt > 0].index
drop_idx

Int64Index([   0,    1,    2,    8,   11,   13,   14,   16,   18,   20,
            ...
            3247, 3252, 3258, 3259, 3260, 3266, 3272, 3273, 3274, 3275],
           dtype='int64', length=1265)

In [10]:
drop_row = data.drop(drop_idx, axis=0)

print(drop_row.shape)
print(data.shape)

(2011, 9)
(3276, 9)


In [11]:
# 빈 컬럼 제거
na_cnt = data.isna().sum()
drop_cols = na_cnt.loc[na_cnt > 0].index

drop_cols

Index(['ph', 'Sulfate', 'Trihalomethanes'], dtype='object')

In [12]:
data = data.drop(drop_cols, axis=1)
data

,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity
0,204.890455,20791.318981,7.300212,564.308654,10.379783,2.963135
1,129.422921,18630.057858,6.635246,592.885359,15.180013,4.500656
2,224.236259,19909.541732,9.275884,418.606213,16.868637,3.055934
3,214.373394,22018.417441,8.059332,363.266516,18.436524,4.628771
4,181.101509,17978.986339,6.546600,398.410813,11.558279,4.075075
...,...,...,...,...,...,...
3271,193.681735,47580.991603,7.166639,526.424171,13.894419,4.435821
3272,193.553212,17329.802160,8.061362,392.449580,19.903225,2.798243
3273,175.762646,33155.578218,7.350233,432.044783,11.039070,3.298875
3274,230.603758,11983.869376,6.303357,402.883113,11.168946,4.708658


### 1.4 Data Split

In [13]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(
    data, label, train_size=0.7, random_state=2021)

print(f"train_data size: {len(train_label)}, {len(train_label)/len(data):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(data):.2f}")

train_data size: 2293, 0.70
test_data size: 983, 0.30


# 2. KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

### 2.1 Best Hyper Parameter
KNeighborsClassifier에서 탐색해야 할 argument
- n_neighbors
    - 몇 개의 이웃으로 예측할 것 인지 설정
- p
    - 거리를 어떤 방식으로 계산할지 설정
    - 1: manhattan distance
    - 2: euclidean distance

### 2.2 Exploration Range Selection

In [15]:
from sklearn.model_selection import GridSearchCV

params = {
    "n_neighbors": [i for i in range(1, 12, 2)],
    "p": [1, 2]
}

params

{'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]}

### 2.3 Exploration

In [16]:
grid_cv = GridSearchCV(knn, param_grid=params, cv=3, n_jobs=-1)

grid_cv.fit(train_data, train_label)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]})

### 2.4 Result

In [17]:
print(f"Best score of paramter search is: {grid_cv.best_score_:.4f}")

Best score of paramter search is: 0.5652


In [18]:
grid_cv.best_params_

{'n_neighbors': 11, 'p': 1}

In [19]:
print("Best parameter of best score is")
print(f"\t n_neighbors: {grid_cv.best_params_['n_neighbors']}")
print(f"\t p: {grid_cv.best_params_['p']}")

Best parameter of best score is
	 n_neighbors: 11
	 p: 1


### 2.5 Predict

In [20]:
train_pred = grid_cv.best_estimator_.predict(train_data)
test_pred = grid_cv.best_estimator_.predict(test_data)

### 2.6 Evalution

In [21]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

print(f"train accuracy is {train_acc:.4f}")
print(f"test accuracy is {test_acc:.4f}")

train accuracy is 0.6520
test accuracy is 0.5595


# 3. Scaling
- KNN은 거리를 기반으로 하는 알고리즘이기 때문에 데이터의 크기에 영향을 받음
- Scaling을 진행해 크기를 맞춰줘야 함

### 3.1 Data Scaling

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_data)

StandardScaler()

In [23]:
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

### 3.2 Exploration

In [24]:
scaling_knn = KNeighborsClassifier()
scaling_grid_cv = GridSearchCV(scaling_knn, param_grid=params, n_jobs=-1)

scaling_grid_cv.fit(scaled_train_data, train_label)

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]})

In [25]:
scaling_grid_cv.best_score_

0.587011825593896

In [26]:
scaling_grid_cv.best_params_

{'n_neighbors': 9, 'p': 1}

### 3.3 Evalution

In [27]:
scaling_train_pred = scaling_grid_cv.best_estimator_.predict(scaled_train_data)
scaling_test_pred = scaling_grid_cv.best_estimator_.predict(scaled_test_data)

scaling_train_acc = accuracy_score(train_label, scaling_train_pred)
scaling_test_acc = accuracy_score(test_label, scaling_test_pred)

print(f"Scaled data train accuracy is {scaling_train_acc:.4f}")
print(f"Scaled data test accuracy is {scaling_test_acc:.4f}")

Scaled data train accuracy is 0.6829
Scaled data test accuracy is 0.5799


# 4. Summary

In [28]:
print(f"test accuracy is {test_acc:.4f}")
print(f"Scaled data test accuracy is {scaling_test_acc:.4f}")

test accuracy is 0.5595
Scaled data test accuracy is 0.5799
